In [23]:
#import necessary libraries
from flask import Flask
from flask_restful import Resource, Api, reqparse
from google.cloud import bigquery
import pandas_gbq
import pandas as pd
import json
import os
from geojson import Feature, FeatureCollection, Point, Polygon
import geopandas
from shapely.geometry import Point, Polygon
import numpy as np

In [24]:
#google credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "Hurricane -32cb24436d46.json"

# #create Flass app instance
# app = Flask(__name__)
# api = Api(app)


#setup connection to BigAssQuery
client = bigquery.Client()


# @app.route("/")
# def index():

query = """
    SELECT year, latitude, longitude, sea_surface_temp
    FROM `bigquery-public-data.noaa_icoads.icoads_core_*` 
    WHERE year >=1851 
    AND latitude <=50
    AND latitude >=0
    AND longitude <=-50
    AND longitude >=-100
    ORDER BY year
"""

# query_res = client.query(query) 

In [25]:
df = pandas_gbq.read_gbq(query)

Downloading: 100%|██████████| 169220630/169220630 [1:32:49<00:00, 30382.92rows/s]


In [26]:
df.to_csv(r'sstAll.csv',index=False)
dfDownload = df

In [45]:
dfDownload=pd.read_csv('sstAll.csv')

In [52]:
dfDownload2=dfDownload
lat_bins=range(0, 51, 10)
dfDownload2['latitudeSquare'] = pd.cut(dfDownload2['latitude'], bins=lat_bins)

In [53]:
long_bins=range(-100, -49, 10)
dfDownload2['longitudeSquare'] = pd.cut(dfDownload2['longitude'], bins=long_bins)
dfDownload2

,year,latitude,longitude,sea_surface_temp,latitudeSquare,longitudeSquare
0,1851,26.60,-89.68,NaN,"(20, 30]","(-90, -80]"
1,1851,8.73,-86.50,NaN,"(0, 10]","(-90, -80]"
2,1851,41.45,-53.33,10.0,"(40, 50]","(-60, -50]"
3,1851,25.87,-77.35,26.1,"(20, 30]","(-80, -70]"
4,1851,25.65,-67.07,NaN,"(20, 30]","(-70, -60]"
...,...,...,...,...,...,...
169220625,2017,29.77,-93.34,NaN,"(20, 30]","(-100, -90]"
169220626,2017,43.81,-83.72,NaN,"(40, 50]","(-90, -80]"
169220627,2017,30.42,-87.83,18.9,"(30, 40]","(-90, -80]"
169220628,2017,29.48,-94.92,22.9,"(20, 30]","(-100, -90]"


In [59]:
dfDownload3=dfDownload2
dfDownload3=dfDownload.dropna()

In [60]:
dfSquare=dfDownload3.groupby(['year','latitudeSquare','longitudeSquare']).agg({'sea_surface_temp': ['min','max','mean','median']})
dfSquare

sea_surface_temp                        
                                                 min   max       mean median
year latitudeSquare longitudeSquare                                         
1851 (0, 10]        (-100, -90]                 19.4  26.7  23.557895   23.3
                    (-90, -80]                  21.7  27.8  25.121739   24.4
                    (-80, -70]                  25.0  28.9  26.983333   26.4
                    (-70, -60]                   NaN   NaN        NaN    NaN
                    (-60, -50]                  26.1  28.3  26.800000   26.4
...                                              ...   ...        ...    ...
2017 (40, 50]       (-100, -90]                 -0.5  24.6  12.544084   13.2
                    (-90, -80]                  -2.5  29.0  15.355935   17.0
                    (-80, -70]                  -1.5  29.4  11.725217   10.4
                    (-70, -60]                  -1.8  28.7   7.306804    6.2
                    (-60, -50]                  -2.0  30.0   5.892106    4.2

[4175 rows x 4 columns]

In [62]:
dfSquare.dropna()
dfProcessed = dfSquare.reset_index()
dfProcessed

year latitudeSquare longitudeSquare sea_surface_temp                   \
                                                       min   max       mean   
0     1851        (0, 10]     (-100, -90]             19.4  26.7  23.557895   
1     1851        (0, 10]      (-90, -80]             21.7  27.8  25.121739   
2     1851        (0, 10]      (-80, -70]             25.0  28.9  26.983333   
3     1851        (0, 10]      (-70, -60]              NaN   NaN        NaN   
4     1851        (0, 10]      (-60, -50]             26.1  28.3  26.800000   
...    ...            ...             ...              ...   ...        ...   
4170  2017       (40, 50]     (-100, -90]             -0.5  24.6  12.544084   
4171  2017       (40, 50]      (-90, -80]             -2.5  29.0  15.355935   
4172  2017       (40, 50]      (-80, -70]             -1.5  29.4  11.725217   
4173  2017       (40, 50]      (-70, -60]             -1.8  28.7   7.306804   
4174  2017       (40, 50]      (-60, -50]             -2.0  30.0   5.892106   

             
     median  
0      23.3  
1      24.4  
2      26.4  
3       NaN  
4      26.4  
...     ...  
4170   13.2  
4171   17.0  
4172   10.4  
4173    6.2  
4174    4.2  

[4175 rows x 7 columns]

In [69]:
dfProcessed.to_csv(r'sstProcessedAll.csv',index=False)
dfProcessed

year latitudeSquare longitudeSquare sea_surface_temp                   \
                                                       min   max       mean   
0     1851        (0, 10]     (-100, -90]             19.4  26.7  23.557895   
1     1851        (0, 10]      (-90, -80]             21.7  27.8  25.121739   
2     1851        (0, 10]      (-80, -70]             25.0  28.9  26.983333   
3     1851        (0, 10]      (-70, -60]              NaN   NaN        NaN   
4     1851        (0, 10]      (-60, -50]             26.1  28.3  26.800000   
...    ...            ...             ...              ...   ...        ...   
4170  2017       (40, 50]     (-100, -90]             -0.5  24.6  12.544084   
4171  2017       (40, 50]      (-90, -80]             -2.5  29.0  15.355935   
4172  2017       (40, 50]      (-80, -70]             -1.5  29.4  11.725217   
4173  2017       (40, 50]      (-70, -60]             -1.8  28.7   7.306804   
4174  2017       (40, 50]      (-60, -50]             -2.0  30.0   5.892106   

             
     median  
0      23.3  
1      24.4  
2      26.4  
3       NaN  
4      26.4  
...     ...  
4170   13.2  
4171   17.0  
4172   10.4  
4173    6.2  
4174    4.2  

[4175 rows x 7 columns]

In [70]:
dfFromCSV = pd.read_csv('sstProcessedAll.csv')
dfFromCSV

,year,latitudeSquare,longitudeSquare,sea_surface_temp,sea_surface_temp.1,sea_surface_temp.2,sea_surface_temp.3
0,NaN,NaN,NaN,min,max,mean,median
1,1851.0,"(0, 10]","(-100, -90]",19.4,26.7,23.557894736842115,23.3
2,1851.0,"(0, 10]","(-90, -80]",21.7,27.8,25.121739130434776,24.4
3,1851.0,"(0, 10]","(-80, -70]",25.0,28.9,26.983333333333338,26.4
4,1851.0,"(0, 10]","(-70, -60]",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
4171,2017.0,"(40, 50]","(-100, -90]",-0.5,24.6,12.544083625820083,13.2
4172,2017.0,"(40, 50]","(-90, -80]",-2.5,29.0,15.355934662905096,17.0
4173,2017.0,"(40, 50]","(-80, -70]",-1.5,29.4,11.725217100185455,10.4
4174,2017.0,"(40, 50]","(-70, -60]",-1.8,28.7,7.306803558346259,6.2


In [71]:
dfDrop=dfFromCSV.dropna()
dfDrop

,year,latitudeSquare,longitudeSquare,sea_surface_temp,sea_surface_temp.1,sea_surface_temp.2,sea_surface_temp.3
1,1851.0,"(0, 10]","(-100, -90]",19.4,26.7,23.557894736842115,23.3
2,1851.0,"(0, 10]","(-90, -80]",21.7,27.8,25.121739130434776,24.4
3,1851.0,"(0, 10]","(-80, -70]",25.0,28.9,26.983333333333338,26.4
5,1851.0,"(0, 10]","(-60, -50]",26.1,28.3,26.8,26.4
6,1851.0,"(10, 20]","(-100, -90]",24.4,28.3,27.38181818181818,27.8
...,...,...,...,...,...,...,...
4171,2017.0,"(40, 50]","(-100, -90]",-0.5,24.6,12.544083625820083,13.2
4172,2017.0,"(40, 50]","(-90, -80]",-2.5,29.0,15.355934662905096,17.0
4173,2017.0,"(40, 50]","(-80, -70]",-1.5,29.4,11.725217100185455,10.4
4174,2017.0,"(40, 50]","(-70, -60]",-1.8,28.7,7.306803558346259,6.2


In [66]:
dfDrop['latitudeSquare'] = dfDrop['latitudeSquare'].str.replace(r'(', '').astype(str)
dfDrop['longitudeSquare'] = dfDrop['longitudeSquare'].str.replace(r'(', '').astype(str)
dfDrop['latitudeSquare'] = dfDrop['latitudeSquare'].str.replace(r']', '').astype(str)
dfDrop['longitudeSquare'] = dfDrop['longitudeSquare'].str.replace(r']', '').astype(str)
dfStrReplace=dfDrop
dfStrReplace

<ipython-input-66-ccbef9c27e58>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfDrop['latitudeSquare'] = dfDrop['latitudeSquare'].str.replace(r'(', '').astype(str)
<ipython-input-66-ccbef9c27e58>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfDrop['longitudeSquare'] = dfDrop['longitudeSquare'].str.replace(r'(', '').astype(str)
<ipython-input-66-ccbef9c27e58>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

,year,latitudeSquare,longitudeSquare,sea_surface_temp,sea_surface_temp.1,sea_surface_temp.2,sea_surface_temp.3
1,1851.0,"0, 10","-100, -90",19.4,26.7,23.557894736842105,23.3
2,1851.0,"0, 10","-90, -80",21.7,27.8,25.12173913043478,24.4
3,1851.0,"0, 10","-80, -70",25.0,28.9,26.983333333333338,26.4
5,1851.0,"0, 10","-60, -50",26.1,28.3,26.8,26.4
6,1851.0,"10, 20","-100, -90",24.4,28.3,27.381818181818186,27.8
...,...,...,...,...,...,...,...
169,1857.0,"30, 40","-70, -60",0.6,29.4,21.113317073170712,21.1
170,1857.0,"30, 40","-60, -50",10.0,28.3,20.839299920445445,20.6
173,1857.0,"40, 50","-80, -70",1.1,26.1,13.636363636363638,14.4
174,1857.0,"40, 50","-70, -60",-2.8,26.7,13.36372332597497,13.9


In [67]:
new = dfStrReplace["latitudeSquare"].str.split(",", n = 1, expand = True) 
dfStrReplace["latitudeLeft"]= new[0] 
dfStrReplace["latitudeRight"]= new[1] 
dfStrReplace

<ipython-input-67-a4e8e887e6a8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfStrReplace["latitudeLeft"]= new[0]
<ipython-input-67-a4e8e887e6a8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfStrReplace["latitudeRight"]= new[1]


,year,latitudeSquare,longitudeSquare,sea_surface_temp,sea_surface_temp.1,sea_surface_temp.2,sea_surface_temp.3,latitudeLeft,latitudeRight
1,1851.0,"0, 10","-100, -90",19.4,26.7,23.557894736842105,23.3,0,10
2,1851.0,"0, 10","-90, -80",21.7,27.8,25.12173913043478,24.4,0,10
3,1851.0,"0, 10","-80, -70",25.0,28.9,26.983333333333338,26.4,0,10
5,1851.0,"0, 10","-60, -50",26.1,28.3,26.8,26.4,0,10
6,1851.0,"10, 20","-100, -90",24.4,28.3,27.381818181818186,27.8,10,20
...,...,...,...,...,...,...,...,...,...
169,1857.0,"30, 40","-70, -60",0.6,29.4,21.113317073170712,21.1,30,40
170,1857.0,"30, 40","-60, -50",10.0,28.3,20.839299920445445,20.6,30,40
173,1857.0,"40, 50","-80, -70",1.1,26.1,13.636363636363638,14.4,40,50
174,1857.0,"40, 50","-70, -60",-2.8,26.7,13.36372332597497,13.9,40,50


In [68]:
dfStrReplace

,year,latitudeSquare,longitudeSquare,sea_surface_temp,sea_surface_temp.1,sea_surface_temp.2,sea_surface_temp.3,latitudeLeft,latitudeRight
1,1851.0,"0, 10","-100, -90",19.4,26.7,23.557894736842105,23.3,0,10
2,1851.0,"0, 10","-90, -80",21.7,27.8,25.12173913043478,24.4,0,10
3,1851.0,"0, 10","-80, -70",25.0,28.9,26.983333333333338,26.4,0,10
5,1851.0,"0, 10","-60, -50",26.1,28.3,26.8,26.4,0,10
6,1851.0,"10, 20","-100, -90",24.4,28.3,27.381818181818186,27.8,10,20
...,...,...,...,...,...,...,...,...,...
169,1857.0,"30, 40","-70, -60",0.6,29.4,21.113317073170712,21.1,30,40
170,1857.0,"30, 40","-60, -50",10.0,28.3,20.839299920445445,20.6,30,40
173,1857.0,"40, 50","-80, -70",1.1,26.1,13.636363636363638,14.4,40,50
174,1857.0,"40, 50","-70, -60",-2.8,26.7,13.36372332597497,13.9,40,50


In [92]:
df

,year,latitudeSquare,longitudeSquare,sea_surface_temp,sea_surface_temp.1,sea_surface_temp.2,sea_surface_temp.3
1,1851.0,"(0, 10]","(-100, -90]",19.4,26.7,23.557894736842115,23.3
2,1851.0,"(0, 10]","(-90, -80]",21.7,27.8,25.121739130434776,24.4
3,1851.0,"(0, 10]","(-80, -70]",25.0,28.9,26.983333333333338,26.4
5,1851.0,"(0, 10]","(-60, -50]",26.1,28.3,26.8,26.4
6,1851.0,"(10, 20]","(-100, -90]",24.4,28.3,27.38181818181818,27.8
...,...,...,...,...,...,...,...
4171,2017.0,"(40, 50]","(-100, -90]",-0.5,24.6,12.544083625820083,13.2
4172,2017.0,"(40, 50]","(-90, -80]",-2.5,29.0,15.355934662905096,17.0
4173,2017.0,"(40, 50]","(-80, -70]",-1.5,29.4,11.725217100185455,10.4
4174,2017.0,"(40, 50]","(-70, -60]",-1.8,28.7,7.306803558346259,6.2


In [129]:
dfNew = df.rename(columns={'sea_surface_temp':'sea_surface_temp_min','sea_surface_temp.1': 'sea_surface_temp_max', 
          'sea_surface_temp.2':'sea_surface_temp_mean', 'sea_surface_temp.3':'sea_surface_temp_median'})

In [130]:
df.to_csv(r'dfNew.csv',index=False)

,year,latitudeSquare,longitudeSquare,sea_surface_temp_min,sea_surface_temp_max,sea_surface_temp_mean,sea_surface_temp_median
1,1851.0,"(0, 10]","(-100, -90]",19.4,26.7,23.557894736842115,23.3
2,1851.0,"(0, 10]","(-90, -80]",21.7,27.8,25.121739130434776,24.4
3,1851.0,"(0, 10]","(-80, -70]",25.0,28.9,26.983333333333338,26.4
5,1851.0,"(0, 10]","(-60, -50]",26.1,28.3,26.8,26.4
6,1851.0,"(10, 20]","(-100, -90]",24.4,28.3,27.38181818181818,27.8
...,...,...,...,...,...,...,...
4171,2017.0,"(40, 50]","(-100, -90]",-0.5,24.6,12.544083625820083,13.2
4172,2017.0,"(40, 50]","(-90, -80]",-2.5,29.0,15.355934662905096,17.0
4173,2017.0,"(40, 50]","(-80, -70]",-1.5,29.4,11.725217100185455,10.4
4174,2017.0,"(40, 50]","(-70, -60]",-1.8,28.7,7.306803558346259,6.2


In [142]:
dfDrop2 = pd.read_csv('dfNew.csv')
dfDrop2= dfNew.dropna()
dfDrop2

,year,latitudeSquare,longitudeSquare,sea_surface_temp_min,sea_surface_temp_max,sea_surface_temp_mean,sea_surface_temp_median
1,1851.0,"(0, 10]","(-100, -90]",19.4,26.7,23.557894736842115,23.3
2,1851.0,"(0, 10]","(-90, -80]",21.7,27.8,25.121739130434776,24.4
3,1851.0,"(0, 10]","(-80, -70]",25.0,28.9,26.983333333333338,26.4
5,1851.0,"(0, 10]","(-60, -50]",26.1,28.3,26.8,26.4
6,1851.0,"(10, 20]","(-100, -90]",24.4,28.3,27.38181818181818,27.8
...,...,...,...,...,...,...,...
4171,2017.0,"(40, 50]","(-100, -90]",-0.5,24.6,12.544083625820083,13.2
4172,2017.0,"(40, 50]","(-90, -80]",-2.5,29.0,15.355934662905096,17.0
4173,2017.0,"(40, 50]","(-80, -70]",-1.5,29.4,11.725217100185455,10.4
4174,2017.0,"(40, 50]","(-70, -60]",-1.8,28.7,7.306803558346259,6.2


In [143]:
def generate_plane(lat_range, lon_range): 
    x=lat_range.replace('(', '').replace(')', '').replace('[', ''). replace(']', '').split(',')
    y=lon_range.replace('(', '').replace(')', '').replace('[', ''). replace(']', '').split(',')
    result=[[lon, lat] for lat in x for lon in y]
    result[-1], result[-2]=result[-2], result[-1]
    result.append(result[0])
    return result

In [144]:
generate_plane('(0, 10]', '(-100, -90]')

[['-100', '0'], [' -90', '0'], [' -90', ' 10'], ['-100', ' 10'], ['-100', '0']]

In [145]:
def df_to_geojson(dfDrop2, properties, lat_col, lon_col):
    geojson={'type': 'FeatureCollection', 'features': []}
    for _, row in dfDrop2.iterrows(): 
#         feature={}
        feature={'type': 'Feature', 
                'properties': {}, 
                'geometry': {'type': 'Polygon', 
                            'coordinates': []}}
        feature['geometry']['coordinates']=[generate_plane(row[lat_col], row[lon_col])]
        for prop in properties: 
            feature['properties'][prop]=row[prop]
        geojson['features'].append(feature)
    return geojson

In [146]:
cols=['year', 
      'sea_surface_temp_min',
      'sea_surface_temp_max', 
      'sea_surface_temp_mean',
      'sea_surface_temp_median']
geojson=df_to_geojson(dfDrop2, cols, 'latitudeSquare', 'longitudeSquare')

In [147]:
geojson

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'year': 1851.0,
    'sea_surface_temp_min': '19.4',
    'sea_surface_temp_max': '26.7',
    'sea_surface_temp_mean': '23.557894736842115',
    'sea_surface_temp_median': '23.3'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[['-100', '0'],
      [' -90', '0'],
      [' -90', ' 10'],
      ['-100', ' 10'],
      ['-100', '0']]]}},
  {'type': 'Feature',
   'properties': {'year': 1851.0,
    'sea_surface_temp_min': '21.7',
    'sea_surface_temp_max': '27.8',
    'sea_surface_temp_mean': '25.121739130434776',
    'sea_surface_temp_median': '24.4'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[['-90', '0'],
      [' -80', '0'],
      [' -80', ' 10'],
      ['-90', ' 10'],
      ['-90', '0']]]}},
  {'type': 'Feature',
   'properties': {'year': 1851.0,
    'sea_surface_temp_min': '25.0',
    'sea_surface_temp_max': '28.9',
    'sea_surface_temp_mean': '26.983333333333338',
    'sea_surface_t

In [148]:
output_filename='sstAll.json'
with open(output_filename, 'w') as output_file: 
    output_file.write('var dataset=')
    json.dump(geojson, output_file, indent=2)